**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

$\epsilon$ is the parameter of our $\epsilon$-greedy policy. It controls the tradeoff between exploration and exploitation. If $\epsilon > 1$ we have a random policy where nothing is learnt. If $\epsilon = 0$ the agent exploits the best option it knows, which might not be the optimal one.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The board has information about the rewards (0.5 for the cheese and -1 for the poison)

The position has information about the rat position (1), the available positions on the board (0) and the walls (-1)

Note : I modified the code to add a wall on the right which was missing.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.5/14.0. Average score (-6.5)
Win/lose count 13.5/14.0. Average score (-3.5)
Win/lose count 8.5/13.0. Average score (-3.8333333333333335)
Win/lose count 9.0/16.0. Average score (-4.625)
Win/lose count 9.0/7.0. Average score (-3.3)
Win/lose count 7.0/12.0. Average score (-3.5833333333333335)
Win/lose count 11.0/12.0. Average score (-3.2142857142857144)
Win/lose count 9.0/20.0. Average score (-4.1875)
Win/lose count 11.5/14.0. Average score (-4.0)
Win/lose count 5.5/9.0. Average score (-3.95)
Win/lose count 8.5/8.0. Average score (-3.5454545454545454)
Win/lose count 10.5/13.0. Average score (-3.4583333333333335)
Win/lose count 11.0/7.0. Average score (-2.8846153846153846)
Win/lose count 15.5/10.0. Average score (-2.2857142857142856)
Win/lose count 10.0/12.0. Average score (-2.2666666666666666)
Win/lose count 13.5/16.0. Average score (-2.28125)
Win/lose count 5.0/12.0. Average score (-2.5588235294117645)
Win/lose count 5.5/9.0. Average score (-2.611111111111111)
Win/lose c

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

PROOF : 

\begin{equation*}Q^\pi(s,a)=E[\sum_{t\ge 0}\gamma^{t}r(s_{t},\pi(a_{t}))|s_{0}=s,a_{0}=a,\pi] \> 
\end{equation*}

\begin{equation*}Q^\pi(s,a)=r(s,a) + E[\sum_{t\ge 1}\gamma^{t}r(s_{t},\pi(a_{t}))|s_{0}=s,a_{0}=a,\pi] \> 
\end{equation*}

\begin{equation*}Q^\pi(s,a)=r(s,a) + \gamma \sum_{s', a'} p(s_{1}=s',a_{1}=a'|s_{0}=s,a_{0}=a)E[\sum_{t\ge 1}\gamma^{t-1}r(s_{t},\pi(a_{t}))|s_{1}=s',a_{1}=a',\pi] \> 
\end{equation*}

\begin{equation*}Q^\pi(s,a)=r(s,a) + \gamma \sum_{s', a'} p(s',a'|s,a) Q^\pi(s',a') \> 
\end{equation*}

\begin{equation*}Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \> 
\end{equation*}

Then,

\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

\begin{equation*}Q^{*}(s,a)=\max_{\pi} r(s,a) + \gamma \sum_{s'} p(s'|s,a) Q^\pi(s',a' = \pi(s') ) \> 
\end{equation*}

\begin{equation*}Q^{*}(s,a)=r(s,a) + \gamma \max_{\pi = (a', \pi')} \sum_{s'} p(s'|s,a) Q^{\pi'}(s',a')  \> 
\end{equation*}

\begin{equation*}Q^{*}(s,a)=r(s,a) + \gamma \sum_{s'} p(s'|s,a) \max_{a', \pi'} Q^{\pi'}(s',a')  \> 
\end{equation*} 

\begin{equation*}Q^{*}(s,a)=r(s,a) + \gamma \sum_{s'} p(s'|s,a) \max_{a'} Q^{*}(s',a')  \> 
\end{equation*} 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Note: I don't know why I had the right to permute the max and the sum

Finally, minimizing the loss

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

is therefore our objective since we want to converge to the optimal Q function

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q = self.model.predict(s.reshape(1, 5, 5, self.n_state)).ravel()
        return np.argmax(q)

    def reinforce(self, state, next_state, action, reward, game_over):
        
        # first : memorize the states

        self.memory.remember([state, next_state, action, reward, game_over])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        # second : learn from the pool
        
        for i in range(self.batch_size):
            ######## FILL IN
            state, next_state, action, reward, game_over = self.memory.random_access()
            input_states[i] = state
            
            target_q[i] = self.model.predict(state.reshape(1, 5, 5, self.n_state))
            
            # update target_q
            if game_over:
                ######## FILL IN
                target_q[i,action] = reward
            else:
                ######## FILL IN
                tmp = self.model.predict(next_state.reshape(1, 5, 5, self.n_state))
                target_q[i,action] = reward + self.discount*max(tmp[0])

        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
        
        
from keras.layers import Flatten, Input
            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(16, activation ='relu')) 
        model.add(Dense(4))
        
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 68        
Total params: 884
Trainable params: 884
Non-trainable params: 0
_________________________________________________________________
Epoch 000/200 | Loss 0.0044 | Win/lose count 3.0/2.0 (1.0)
Epoch 001/200 | Loss 0.0043 | Win/lose count 5.0/6.0 (-1.0)
Epoch 002/200 | Loss 0.0087 | Win/lose count 6.5/5.0 (1.5)
Epoch 003/200 | Loss 0.0085 | Win/lose count 3.5/1.0 (2.5)
Epoch 004/200 | Loss 0.0403 | Win/lose count 7.0/4.0 (3.0)
Epoch 005/200 | Loss 0.0782 | Win/lose count 5.0/4.0 (1.0)
Epoch 006/200 | Loss 0.0055 | Win/lose count 3.5/3.0 

Epoch 127/200 | Loss 0.0047 | Win/lose count 10.5/2.0 (8.5)
Epoch 128/200 | Loss 0.0045 | Win/lose count 6.0/0 (6.0)
Epoch 129/200 | Loss 0.0040 | Win/lose count 5.5/1.0 (4.5)
Epoch 130/200 | Loss 0.0032 | Win/lose count 6.0/4.0 (2.0)
Epoch 131/200 | Loss 0.0010 | Win/lose count 8.0/1.0 (7.0)
Epoch 132/200 | Loss 0.0037 | Win/lose count 6.0/4.0 (2.0)
Epoch 133/200 | Loss 0.0488 | Win/lose count 19.0/3.0 (16.0)
Epoch 134/200 | Loss 0.0570 | Win/lose count 10.0/1.0 (9.0)
Epoch 135/200 | Loss 0.0041 | Win/lose count 13.0/2.0 (11.0)
Epoch 136/200 | Loss 0.0021 | Win/lose count 9.0/2.0 (7.0)
Epoch 137/200 | Loss 0.0034 | Win/lose count 7.0/1.0 (6.0)
Epoch 138/200 | Loss 0.0034 | Win/lose count 11.5/6.0 (5.5)
Epoch 139/200 | Loss 0.0034 | Win/lose count 7.5/1.0 (6.5)
Epoch 140/200 | Loss 0.0022 | Win/lose count 5.0/2.0 (3.0)
Epoch 141/200 | Loss 0.0024 | Win/lose count 11.5/2.0 (9.5)
Epoch 142/200 | Loss 0.0020 | Win/lose count 9.0/4.0 (5.0)
Epoch 143/200 | Loss 0.0030 | Win/lose count 11.5/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(16, (3,3), input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Conv2D(8, (3,3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 3, 16)          304       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 36        
Total params: 1,500
Trainable params: 1,500
Non-trainable params: 0
_________________________________________________________________
Epoch 000/200 | Loss 0.0131 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/200 | Loss 0.0006 | Win/lose count 4.0/2.0 (2.0)
Epoch 002/200 | Loss 0.0024 | Win/lose count 2.5/4.0 (-1.5)
Epoch 003/200 | Loss 0.0018 | Win/lose count 3.5/9.0 (-5.5)
Epoch 004/200 | Loss 0.0010 | Win

Epoch 124/200 | Loss 0.0016 | Win/lose count 18.5/8.0 (10.5)
Epoch 125/200 | Loss 0.0012 | Win/lose count 18.5/6.0 (12.5)
Epoch 126/200 | Loss 0.0013 | Win/lose count 11.0/4.0 (7.0)
Epoch 127/200 | Loss 0.0488 | Win/lose count 17.5/4.0 (13.5)
Epoch 128/200 | Loss 0.0040 | Win/lose count 15.0/3.0 (12.0)
Epoch 129/200 | Loss 0.0012 | Win/lose count 11.0/2.0 (9.0)
Epoch 130/200 | Loss 0.0007 | Win/lose count 17.5/4.0 (13.5)
Epoch 131/200 | Loss 0.0554 | Win/lose count 11.5/1.0 (10.5)
Epoch 132/200 | Loss 0.0014 | Win/lose count 13.0/0 (13.0)
Epoch 133/200 | Loss 0.0089 | Win/lose count 18.0/1.0 (17.0)
Epoch 134/200 | Loss 0.0035 | Win/lose count 12.5/3.0 (9.5)
Epoch 135/200 | Loss 0.0561 | Win/lose count 13.5/1.0 (12.5)
Epoch 136/200 | Loss 0.0029 | Win/lose count 18.0/4.0 (14.0)
Epoch 137/200 | Loss 0.0505 | Win/lose count 20.0/2.0 (18.0)
Epoch 138/200 | Loss 0.0075 | Win/lose count 10.0/5.0 (5.0)
Epoch 139/200 | Loss 0.0019 | Win/lose count 21.5/4.0 (17.5)
Epoch 140/200 | Loss 0.0011 | 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 3, 16)          304       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_3 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 36        
Total params: 1,500
Trainable params: 1,500
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_5 

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

The main issue is the lack of exploration of our agent, which often gets stuck in loops going back and forth between two states. This behaviour doesn't provide any rewards at all, so the agent doesn't get any indication that there might be some cheese elsewhere.

The sparser the environement (i.e. the lower the temperature), the more likely it is to get stuck in a loop.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent, env, epoch, prefix='', epsilon_decay=0.5):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            agent.set_epsilon(agent.epsilon*epsilon_decay)

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size, grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 3, 3, 16)          448       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_5 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 36        
Total params: 1,644
Trainable params: 1,644
Non-trainable params: 0
_________________________________________________________________
Epoch 000/200 | Loss 0.0031 | Win/lose count 5.0/28.6000000000001 (-23.6000000000001)
Epoch 001/200 | Loss 0.0085 | Win/lose count 4.0/21.40000000000004 (-17.40000000000004)
Epoch 002/200 | Loss 0.0157 | Win/lose count 3.0/24.50000000000006 (-21.50000000000006)
Epoch 003/

Epoch 083/200 | Loss 0.0073 | Win/lose count 17.0/14.299999999999965 (2.700000000000035)
Epoch 084/200 | Loss 0.0512 | Win/lose count 21.5/13.09999999999997 (8.40000000000003)
Epoch 085/200 | Loss 0.0434 | Win/lose count 12.5/14.799999999999963 (-2.2999999999999634)
Epoch 086/200 | Loss 0.0065 | Win/lose count 17.0/15.59999999999996 (1.4000000000000394)
Epoch 087/200 | Loss 0.0028 | Win/lose count 17.0/12.199999999999973 (4.800000000000027)
Epoch 088/200 | Loss 0.0063 | Win/lose count 12.0/14.799999999999963 (-2.7999999999999634)
Epoch 089/200 | Loss 0.0087 | Win/lose count 19.0/13.899999999999967 (5.100000000000033)
Epoch 090/200 | Loss 0.0060 | Win/lose count 19.5/13.899999999999967 (5.600000000000033)
Epoch 091/200 | Loss 0.0081 | Win/lose count 18.0/14.199999999999966 (3.8000000000000345)
Epoch 092/200 | Loss 0.0036 | Win/lose count 18.0/11.699999999999974 (6.300000000000026)
Epoch 093/200 | Loss 0.0114 | Win/lose count 7.0/18.099999999999987 (-11.099999999999987)
Epoch 094/200 | L

Epoch 175/200 | Loss 0.0034 | Win/lose count 18.0/14.299999999999965 (3.700000000000035)
Epoch 176/200 | Loss 0.0087 | Win/lose count 23.0/11.39999999999998 (11.60000000000002)
Epoch 177/200 | Loss 0.0532 | Win/lose count 26.0/7.39999999999999 (18.60000000000001)
Epoch 178/200 | Loss 0.0065 | Win/lose count 21.5/12.599999999999973 (8.900000000000027)
Epoch 179/200 | Loss 0.0158 | Win/lose count 15.5/14.099999999999966 (1.400000000000034)
Epoch 180/200 | Loss 0.0068 | Win/lose count 15.5/14.499999999999964 (1.0000000000000355)
Epoch 181/200 | Loss 0.0112 | Win/lose count 18.0/13.999999999999968 (4.000000000000032)
Epoch 182/200 | Loss 0.0054 | Win/lose count 26.0/11.199999999999983 (14.800000000000017)
Epoch 183/200 | Loss 0.0063 | Win/lose count 18.0/13.899999999999968 (4.100000000000032)
Epoch 184/200 | Loss 0.0076 | Win/lose count 18.0/13.199999999999976 (4.800000000000024)
Epoch 185/200 | Loss 0.0064 | Win/lose count 9.0/16.699999999999967 (-7.699999999999967)
Epoch 186/200 | Loss 0

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 23.0/2.0. Average score (21.0)
Win/lose count 18.0/0. Average score (19.5)
Win/lose count 18.5/1.0. Average score (18.833333333333332)
Win/lose count 23.0/2.0. Average score (19.375)
Win/lose count 21.5/2.0. Average score (19.4)
Win/lose count 22.0/0. Average score (19.833333333333332)
Win/lose count 20.5/2.0. Average score (19.642857142857142)
Win/lose count 27.0/1.0. Average score (20.4375)
Win/lose count 17.5/1.0. Average score (20.0)
Win/lose count 17.0/0. Average score (19.7)
Win/lose count 11.0/1.0. Average score (18.818181818181817)
Win/lose count 23.5/3.0. Average score (18.958333333333332)
Win/lose count 19.5/1.0. Average score (18.923076923076923)
Win/lose count 18.0/3.0. Average score (18.642857142857142)
Win/lose count 4.5/1.0. Average score (17.633333333333333)
Win/lose count 14.0/0. Average score (17.40625)
Win/lose count 8.0/0. Average score (16.852941176470587)
Win/lose count 21.0/1.0. Average score (17.02777777777778)
Win/lose count 23.5/0. Average score

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***